Chapter 7. Quantum Variational Optimaization and adiabatic MEthods


In [2]:
import cirq
print('cirq version',cirq.__version__)
import numpy as np
print('numpy version',np.__version__)
import scipy
from scipy.optimize import minimize
print('scipy version',scipy.__version__)


def setup_vqe(hamiltonian_bases=['ZZZ'], hamiltonian_scales=[-1.0]):

    num_qubits = len(hamiltonian_bases[0])
    eigen_values_dict = {}

    for base,scale in zip(hamiltonian_bases,hamiltonian_scales):
        eigen_values = []
        for i, char in enumerate(base):
            if char == 'Z':
                eigens = np.array([1, -1])
            elif char == 'I':
                eigens = np.array([1, 1])
            else:
                raise NotImplementedError(f"The Gate {char} is yet to be implemented")

            if len(eigen_values) == 0:
                eigen_values = eigens
            else:
                eigen_values = np.outer(eigen_values, eigens).flatten()

        eigen_values_dict_elem = {}

        for i, x in enumerate(list(eigen_values)):
            eigen_values_dict_elem[i] = scale * x

        eigen_values_dict[base] = eigen_values_dict_elem


    return eigen_values_dict, num_qubits


def ansatz_parameterized(theta,num_qubits=3):
    """
    Create an Ansatz
    :param theta: 
    :param num_qubits: 
    :return: 
    """
    qubits = [cirq.LineQubit(c) for c in range(num_qubits)]
    circuit = cirq.Circuit()
    for i in range(num_qubits):
        circuit.append(cirq.ry(theta[i]*np.pi)(qubits[i]))
    circuit.append(cirq.measure(*qubits, key='m'))
    print(circuit)
    return circuit, qubits


def compute_expectation(circuit, eigen_value_dict={}, copies=10000) -> float:
    sim = cirq.Simulator()
    results = sim.run(circuit, repetitions=copies)
    output = dict(results.histogram(key='m'))
    print('Stats', output)
    _expectation_ = 0
    for base in list(eigen_value_dict.keys()):
        for i in list(output.keys()):
            _expectation_ += eigen_value_dict[base][i] * output[i]

    _expectation_ = _expectation_ / copies

    return _expectation_


def VQE_routine(hamiltonian_bases=['ZZZ'], hamiltonian_scales=[1.], copies=1000,
         initial_theta=[0.5, 0.5, 0.5], verbose=True):
    eigen_value_dict, num_qubits = setup_vqe(hamiltonian_bases=hamiltonian_bases,
                                             hamiltonian_scales=hamiltonian_scales)
    print(eigen_value_dict)
    initial_theta = np.array(initial_theta)

    def objective(theta):
        circuit, qubits = ansatz_parameterized(theta, num_qubits)
        expectation = compute_expectation(circuit, eigen_value_dict, copies)
        if verbose:
            print(f" Theta: {theta} Expectation: {expectation}")
        return expectation

    result = minimize(objective, x0=initial_theta, method='COBYLA')
    print(result)
    circuit, _ = ansatz_parameterized(result.x, num_qubits)
    sim = cirq.Simulator()
    results = sim.run(circuit, repetitions=copies)
    stats = dict(results.histogram(key='m'))
    return result.x, result.fun, stats


if __name__ == '__main__':
    optim_theta, optim_func, hist_stats = VQE_routine(hamiltonian_bases=['IIZZ','IZIZ','IZZI','ZIIZ','ZZII'], hamiltonian_scales=[0.5,.5,.50,.50,.50],
                                          initial_theta=[0.5, 0.5,0.5,0.5])
    print(f"VQE Results: Minimum Hamiltonian Energy:{optim_func} at theta: {optim_theta}")
    print(f"Histogram for optimized State:", hist_stats)
    

cirq version 1.6.1
numpy version 2.3.0
scipy version 1.16.2
{'IIZZ': {0: np.float64(0.5), 1: np.float64(-0.5), 2: np.float64(-0.5), 3: np.float64(0.5), 4: np.float64(0.5), 5: np.float64(-0.5), 6: np.float64(-0.5), 7: np.float64(0.5), 8: np.float64(0.5), 9: np.float64(-0.5), 10: np.float64(-0.5), 11: np.float64(0.5), 12: np.float64(0.5), 13: np.float64(-0.5), 14: np.float64(-0.5), 15: np.float64(0.5)}, 'IZIZ': {0: np.float64(0.5), 1: np.float64(-0.5), 2: np.float64(0.5), 3: np.float64(-0.5), 4: np.float64(-0.5), 5: np.float64(0.5), 6: np.float64(-0.5), 7: np.float64(0.5), 8: np.float64(0.5), 9: np.float64(-0.5), 10: np.float64(0.5), 11: np.float64(-0.5), 12: np.float64(-0.5), 13: np.float64(0.5), 14: np.float64(-0.5), 15: np.float64(0.5)}, 'IZZI': {0: np.float64(0.5), 1: np.float64(0.5), 2: np.float64(-0.5), 3: np.float64(-0.5), 4: np.float64(-0.5), 5: np.float64(-0.5), 6: np.float64(0.5), 7: np.float64(0.5), 8: np.float64(0.5), 9: np.float64(0.5), 10: np.float64(-0.5), 11: np.float64(-

In [5]:
if __name__ == '__main__':
    optim_theta, optim_func,hist_stats = VQE_routine(hamiltonian_bases=['ZZ'],
        hamiltonian_scales=[-1.0],
        initial_theta=[0.75,0.75])
    print(f"VQE Results: Minimum Hamiltonian Energy:{optim_func} at theta: {optim_theta}")
    print(f"Histogram for optimized State:", hist_stats)

{'ZZ': {0: np.float64(-1.0), 1: np.float64(1.0), 2: np.float64(1.0), 3: np.float64(-1.0)}}
0: ───Ry(0.75π)───M('m')───
                  │
1: ───Ry(0.75π)───M────────
Stats {3: 720, 1: 110, 2: 146, 0: 24}
 Theta: [0.75 0.75] Expectation: -0.488
0: ───Ry(1.75π)───M('m')───
                  │
1: ───Ry(0.75π)───M────────
Stats {1: 732, 3: 119, 0: 129, 2: 20}
 Theta: [1.75 0.75] Expectation: 0.504
0: ───Ry(0.75π)───M('m')───
                  │
1: ───Ry(1.75π)───M────────
Stats {2: 708, 3: 134, 0: 134, 1: 24}
 Theta: [0.75 1.75] Expectation: 0.464
0: ───Ry(0.028π)───M('m')───
                   │
1: ───Ry(0.058π)───M────────
Stats {0: 994, 1: 4, 2: 2}
 Theta: [0.02849641 0.0575893 ] Expectation: -0.988
0: ───Ry(0.254π)────M('m')───
                    │
1: ───Ry(-0.917π)───M────────
Stats {1: 820, 3: 154, 0: 23, 2: 3}
 Theta: [ 0.25425291 -0.91659446] Expectation: 0.646
0: ───Ry(-0.389π)───M('m')───
                    │
1: ───Ry(0.333π)────M────────
Stats {0: 495, 2: 241, 1: 179, 3: 85}


In [6]:
if __name__ == '__main__':
    optim_theta, optim_func,hist_stats = VQE_routine(hamiltonian_bases=['ZZ','ZI'], hamiltonian_scales=[-1.0,-1.0], initial_theta=[0.5, 0.5])
print(f"VQE Results: Minimum Hamiltonian Energy:{optim_func} at theta: {optim_theta}")
print(f"Histogram for optimized State:", hist_stats)

{'ZZ': {0: np.float64(-1.0), 1: np.float64(1.0), 2: np.float64(1.0), 3: np.float64(-1.0)}, 'ZI': {0: np.float64(-1.0), 1: np.float64(-1.0), 2: np.float64(1.0), 3: np.float64(1.0)}}
0: ───Ry(0.5π)───M('m')───
                 │
1: ───Ry(0.5π)───M────────
Stats {2: 237, 1: 255, 3: 253, 0: 255}
 Theta: [0.5 0.5] Expectation: -0.036
0: ───Ry(1.5π)───M('m')───
                 │
1: ───Ry(0.5π)───M────────
Stats {2: 264, 1: 278, 0: 254, 3: 204}
 Theta: [1.5 0.5] Expectation: 0.02
0: ───Ry(0.5π)───M('m')───
                 │
1: ───Ry(1.5π)───M────────
Stats {2: 248, 3: 259, 0: 264, 1: 229}
 Theta: [0.5 1.5] Expectation: -0.032
0: ───Ry(-0.497π)───M('m')───
                    │
1: ───Ry(0.429π)────M────────
Stats {0: 301, 2: 307, 3: 186, 1: 206}
 Theta: [-0.4974587   0.42875295] Expectation: 0.012
0: ───Ry(0.998π)───M('m')───
                   │
1: ───Ry(0.459π)───M────────
Stats {3: 432, 2: 568}
 Theta: [0.99830172 0.45882485] Expectation: 1.136
0: ───Ry(0.49π)────M('m')───
               

Listing 7-2 Max-Cut clustering


In [7]:
import cirq
from vqe_cirq import *
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt


class QuantumMaxCutClustering:

    def __init__(self, adjacency_matrix: np.ndarray, invert_adjacency=True):
        self.adjacency_matrix = adjacency_matrix
        self.num_vertices = self.adjacency_matrix.shape[0]
        self.hamiltonian_basis_template = 'I' * self.num_vertices
        if invert_adjacency:
            self.hamiltonian = 1 - self.adjacency_matrix
        else:
            self.hamiltonian = self.adjacency_matrix

    def create_max_cut_hamiltonian(self):

        hamiltonian_bases, hamiltonian_coefficients = [], []
        for i in range(self.num_vertices):
            for j in range(i + 1, self.num_vertices):
                if self.hamiltonian[i, j] > 0:
                    hamiltonian_coefficients.append(self.hamiltonian[i, j])

                    hamiltonian_base = ''
                    for k, c in enumerate(self.hamiltonian_basis_template):
                        if k in [i, j]:
                            hamiltonian_base += 'Z'
                        else:
                            hamiltonian_base += self.hamiltonian_basis_template[k]
                    hamiltonian_bases.append(hamiltonian_base)
        return hamiltonian_bases, hamiltonian_coefficients

    def vqe_simulation(self, hamiltonian_bases,
                       hamiltonian_coefficients,
                       initial_theta=None,
                       copies=10000):
        if initial_theta is None:
            initial_theta = [0.5] * self.num_vertices
        optim_theta, optim_func, hist_stats = \
            VQE_routine(hamiltonian_bases=hamiltonian_bases,
                        hamiltonian_scales=hamiltonian_coefficients,
                        initial_theta=initial_theta,
                        copies=copies)
        solution_stat = max(hist_stats, key=hist_stats.get)
        solution_stat = bin(solution_stat).replace("0b", "")
        solution_stat = (self.num_vertices - len(solution_stat)) * "0" + solution_stat

        return optim_theta, optim_func, hist_stats, solution_stat

    def max_cut_cluster(self, distance_matrix, solution_state):
        print(distance_matrix)
        G = nx.Graph()
        G.add_nodes_from(np.arange(0, self.num_vertices, 1))
        edge_list = []
        for i in range(self.num_vertices):
            for j in range(i + 1, self.num_vertices):
                if distance_matrix[i, j] > 0:
                    edge_list.append((i, j, 1.0))
        G.add_weighted_edges_from(edge_list)
        colors = []
        for s in solution_state:
            if int(s) == 1:
                colors.append('r')
            else:
                colors.append('b')
        pos = nx.spring_layout(G)
        default_axes = plt.axes(frameon=True)
        nx.draw_networkx(G, node_color=colors, node_size=600, alpha=.8, ax=default_axes, pos=pos)
        plt.savefig('Maxcut_clustering.png')
        

    def main(self):
        hamiltonian_bases, hamiltonian_coefficients = self.create_max_cut_hamiltonian()
        print(hamiltonian_bases)
        optim_theta, optim_func, \
        hist_stats, solution_state = self.vqe_simulation(hamiltonian_bases,
                                                         hamiltonian_coefficients)

        print(f"VQE Results: Minimum Hamiltonian Energy:{optim_func} at theta: {optim_theta}")
        print(f"Histogram for optimized State:", hist_stats)
        print(f"Solution state: {solution_state}")
        self.max_cut_cluster(distance_matrix=self.hamiltonian, solution_state=solution_state)


if __name__ == '__main__':
    adjacency_matrix = np.array([[0, 0, 0, 0],
                                 [0, 0, 0, 1],
                                 [0, 0, 0, 0],
                                 [0, 1, 0, 0]])
    mc = QuantumMaxCutClustering(adjacency_matrix=adjacency_matrix)
    mc.main()

ModuleNotFoundError: No module named 'vqe_cirq'

In [8]:
import cirq
import numpy as np
print('cirq version',cirq.__version__)
print('np version', np.__version__)




class QAOA:

    def __init__(self, num_elems:int,
                 hamiltonian_type:str,
                 hamiltonian_interactions:np.ndarray,
                 verbose=True):
        self.num_elems = num_elems
        self.hamiltonian_type = hamiltonian_type
        self.hamiltonian_interactions = hamiltonian_interactions
        self.verbose = verbose
        if self.hamiltonian_type not in ['isling']:
            raise ValueError(f"No support for the Hamiltonian type {self.hamiltonian_type}")
        self.qubits = [cirq.LineQubit(x) for x in range(num_elems)]


    @staticmethod
    def interaction_gate(q1, q2, gamma=1):
        circuit = cirq.Circuit()
        circuit.append(cirq.CZ(q1, q2)**gamma)
        circuit.append([cirq.X(q2), cirq.CZ(q1, q2)**(-gamma), cirq.X(q2)])
        circuit.append([cirq.X(q1), cirq.CZ(q1, q2) **(-gamma), cirq.X(q1)])
        circuit.append([cirq.X(q1), cirq.X(q2), cirq.CZ(q1, q2) ** gamma, cirq.X(q1), cirq.X(q2)])
        return circuit

# Build the Target Hamiltonian based circuit Evolution
    def target_hamiltonian_evolution_circuit(self, gamma):
        circuit = cirq.Circuit()
        # Apply the interaction gates to all the qubit pairs

        for i in range(self.num_elems):

            for j in range(i+1, self.num_elems):
                circuit.append(self.interaction_gate(
                                    self.qubits[i], self.qubits[j],
                                    gamma=gamma))
        return circuit

# Build the Starting Hamiltonian based evolution circuit
    def starting_hamiltonian_evolution_circuit(self, beta):
        for i in range(self.num_elems):
            yield cirq.X(self.qubits[i])**beta

    def build_qoaa_circuit(self, gamma_store, beta_store):
        self.circuit = cirq.Circuit()
        # Hadamard gate on each qubit to get an equal superposition state
        print(self.qubits)
        self.circuit.append(cirq.H.on_each(self.qubits))

        for i in range(len(gamma_store)):
            self.circuit.append(self.target_hamiltonian_evolution_circuit(gamma_store[i]))
            self.circuit.append(self.starting_hamiltonian_evolution_circuit(beta_store[i]))

    def simulate(self):
        #print(self.circuit)
        sim = cirq.Simulator()
        waveform = sim.simulate(self.circuit)
        return waveform


    def expectation(self,waveform):

        expectation = 0
        prob_from_waveform = (np.absolute(waveform.final_state))**2
        #print(prob_from_waveform)
        for i in range(len(prob_from_waveform)):
            base = bin(i).replace("0b", "")
            base = (self.num_elems - len(base))*'0' + base
            base_array = []
            for b in base:
                if int(b) == 0:
                    base_array.append(-1)
                else:
                    base_array.append(1)

            base_array = np.array(base_array)
            base_interactions = np.outer(base_array, base_array)
            #print(i, prob_from_waveform[i], np.sum(np.multiply(base_interactions,self.hamiltonian_interactions)))
            expectation =+ prob_from_waveform[i]*np.sum(np.multiply(base_interactions,self.hamiltonian_interactions))
        return expectation

    def optimize_params(self, gammas, betas, verbose=True):
        expectation_dict = {}
        waveforms_dict = {}
        for i, gamma in enumerate(gammas):
            for j, beta in enumerate(betas):
                self.build_qoaa_circuit([gamma],[beta])
                waveform = self.simulate()
                expectation = self.expectation(waveform)
                expectation_dict[(gamma,beta)] = expectation
                waveforms_dict[(gamma,beta)] = waveform.final_state
                if verbose:
                    print(f"Expectation for gamma:{gamma}, beta:{beta} = {expectation}")
        return expectation_dict, waveforms_dict


    def main(self):
        gammas = np.linspace(0, 1,50)
        betas = np.linspace(0, np.pi, 50)
        expectation_dict, waveform_dict = self.optimize_params(gammas, betas)
        expectation_vals = np.array(list(expectation_dict.values()))
        expectation_params = list(expectation_dict.keys())
        waveform_vals = np.array(list(waveform_dict.values()))
        optim_param = expectation_params[np.argmin(expectation_vals)]
        optim_expectation = expectation_vals[np.argmin(expectation_vals)]
        optim_waveform = waveform_vals[np.argmin(expectation_vals)]
        print(f"Optimized parameters")
        print(f"-----------------------------")
        print(f"  gamma,beta = {optim_param[0]}, {optim_param[1]}")
        print(f"  Expectation = {optim_expectation}")
        print(f"-----------------------------")
        optimal_waveform_prob = np.array([np.abs(x)**2 for x in optim_waveform])
        states = np.array([bin(i).replace('0b', "") for i in range(2 ** self.num_elems)])
        states = np.array([((self.num_elems - len(s)) * '0' + s) for s in states])
        sort_indices = np.argsort(-1 * np.array(optimal_waveform_prob))
        print(sort_indices)
        optimal_waveform_prob = optimal_waveform_prob[sort_indices]
        states = states[sort_indices]
        print(f"State | Probability")
        print(f"-----------------------------")
        for i in range(len(states)):
            print(f"{states[i]} | {np.round(optimal_waveform_prob[i],3)}")
            print(f"-----------------------------")

        return expectation_dict


if __name__ == '__main__':
    hamiltonian_interaction = np.array([[0,-1,-1,-1],
                                        [0,0,-1,-1],
                                        [0,0,0,-1],
                                        [0,0,0,0]])
    qaoa_obj = QAOA(num_elems=4,
                    hamiltonian_type='isling',
                    hamiltonian_interactions=hamiltonian_interaction)
    expectation_dict = qaoa_obj.main()
    

cirq version 1.6.1
np version 2.3.0
[cirq.LineQubit(0), cirq.LineQubit(1), cirq.LineQubit(2), cirq.LineQubit(3)]


AttributeError: 'StateVectorTrialResult' object has no attribute 'final_state'

In [9]:
import cirq
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import networkx as nx
print('cirq version', cirq.__version__)
print('numpy version', np.__version__)
print('matplotlib version', matplotlib.__version__)
print('networkx version', nx.__version__)


class GraphQuantumRandomWalk:

    def __init__(self, graph_hamiltonian, t, verbose=True):
        self.graph_ham = graph_hamiltonian
        self.num_vertices = self.graph_ham.shape[0]
        self.num_qubits = int(np.log2(self.num_vertices))
        self.qubits = [cirq.LineQubit(i) for i in range(self.num_qubits)]
        self.t = t
        self.verbose = verbose

    @staticmethod
    def diagonal_exponential(qubits, eigen_vals, t):
        circuit = cirq.Circuit()
        q1 = qubits[0]
        q2 = qubits[1]
        circuit.append(cirq.CZ(q1, q2) ** (-eigen_vals[-1] * t / np.pi))
        circuit.append([cirq.X(q2), cirq.CZ(q1, q2) ** (-eigen_vals[-2] * t / np.pi), cirq.X(q2)])
        circuit.append([cirq.X(q1), cirq.CZ(q1, q2) ** (-eigen_vals[-3] * t / np.pi), cirq.X(q1)])
        circuit.append(
            [cirq.X(q1), cirq.X(q2), cirq.CZ(q1, q2) ** (-eigen_vals[-4] * t / np.pi), cirq.X(q1), cirq.X(q2)])
        return circuit

    def unitary(self):
        eigen_vals, eigen_vecs = np.linalg.eigh(self.graph_ham)
        idx = eigen_vals.argsort()[::-1]
        eigen_vals = eigen_vals[idx]
        eigen_vecs = eigen_vecs[:, idx]
        if self.verbose:
            print(f"The Eigen values: {eigen_vals}")

        self.circuit = cirq.Circuit()
        self.circuit.append(cirq.H.on_each(self.qubits))
        self.circuit += self.diagonal_exponential(self.qubits, eigen_vals, self.t)
        self.circuit.append(cirq.H.on_each(self.qubits))

    def simulate(self):
        sim = cirq.Simulator()
        results = sim.simulate(self.circuit).final_state
        prob_dist = [np.abs(a) ** 2 for a in results]
        return prob_dist

    def main(self):
        self.unitary()
        prob_dist = self.simulate()
        if self.verbose:
            print(f"The converged prob_dist: {prob_dist}")
        return prob_dist


if __name__ == '__main__':
    graph_hamiltonian = np.ones((4, 4))
    #graph_hamiltonian = np.array([[1, 1, 1, 0], [1, 1, 0, 1], [1, 0, 1, 1], [0, 1, 1, 1]])
    time_to_simulate = 4
    steps = 80
    time_trace = []
    prob_dist_trace = []
    for t in np.linspace(0, time_to_simulate):
        gqrq = GraphQuantumRandomWalk(graph_hamiltonian=graph_hamiltonian, t=t)
        prob_dist = gqrq.main()
        time_trace.append(t)
        prob_dist_trace.append(prob_dist)
    prob_dist_trace = np.array(prob_dist_trace)
    plt.plot(time_trace, prob_dist_trace[:, 0])
    plt.show()
    rows, cols = np.where(graph_hamiltonian == 1)
    edges = zip(rows.tolist(), cols.tolist())
    gr = nx.Graph()
    gr.add_edges_from(edges)
    nx.draw(gr,node_size=4)
    plt.show()

cirq version 1.6.1
numpy version 2.3.0
matplotlib version 3.10.3
networkx version 3.6.1
The Eigen values: [ 4.00000000e+00 -1.23259516e-32 -3.42450962e-16 -9.89816667e-16]


AttributeError: 'StateVectorTrialResult' object has no attribute 'final_state'